In [3]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

import torch
torch.manual_seed(42)
from google.colab import drive
drive.mount('/content/drive')

data_path = 'drive/MyDrive/MLE/protein_data/'

GPU = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
df = pd.read_parquet(data_path + 'train_protbert_embeddings.parquet').drop(columns=['numpy_embeddings'])
df.head(1)

,family_id,sequence_name,family_accession,aligned_sequence,sequence
8,NBD_C,R5HG95_9SPIR/261-424,PF17042.5,FVVSGSATQITANQIEKLEEADEF.ENL.LVI......N.LDMKTV...,FVVSGSATQITANQIEKLEEADEFENLLVINLDMKTVLAGVSDEIV...


In [90]:
# Load protbert model
!pip install transformers
from transformers import BertForMaskedLM, BertTokenizer, pipeline
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertForMaskedLM.from_pretrained("Rostlab/prot_bert")
if GPU:
    model.cuda()
import torch
import re


x_aminos = re.compile("[UZOB]")

def prepare_input(seq, **tokenizer_args):
    seq = x_aminos.sub("X", ' '.join(seq))
    input_ids = tokenizer.encode(seq, add_special_tokens=True, **tokenizer_args)
    return input_ids

def get_embeddings(seq):
    input_ids = prepare_input(seq)
    input_ids = torch.tensor([input_ids], device='cuda' if GPU else 'cpu')
    with torch.no_grad():
        return model(input_ids)

eg_input = get_embeddings("RRWWRRRRW")
eg_input

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


MaskedLMOutput(loss=None, logits=tensor([[[-1.8532e+01, -1.9899e+01, -1.9202e+01, -1.9233e+01, -2.1818e+01,
          -3.3182e-01, -4.3617e-01, -1.6150e-01, -2.5577e-02, -1.2855e+00,
           4.6538e-01, -1.3698e+00,  2.5461e-02,  1.3961e+00, -2.3760e+00,
          -2.5162e-01, -6.2316e-01, -9.2691e-01, -8.8104e-01, -1.4678e+00,
          -1.6217e+00,  3.2145e+00, -2.0604e+00, -4.5067e-01,  4.5485e-01,
          -3.9251e+00, -1.8108e+01, -1.8154e+01, -1.8443e+01, -1.8931e+01],
         [-2.1589e+01, -2.1636e+01, -2.1418e+01, -2.1295e+01, -2.2876e+01,
          -8.4695e-01, -8.1242e-01,  7.8904e-01, -1.7455e+00, -1.4653e+00,
           5.6490e-01, -1.1674e+00,  1.2356e+00,  4.1438e+00, -2.4756e+00,
           2.0495e+00, -1.3056e+00,  7.8865e-01,  1.2412e+00, -2.7057e+00,
          -1.4608e+00, -3.3682e-01, -2.7666e+00,  2.4259e-01,  3.9161e+00,
          -6.9252e+00, -2.1043e+01, -2.1170e+01, -2.0820e+01, -2.1201e+01],
         [-2.1355e+01, -2.0936e+01, -2.0803e+01, -1.9996e+01, -2.

In [7]:
## import libraries

#PyTorch 

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as torch_optim
from torchvision import models
from torch.nn import init
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils import data
from torch.optim import lr_scheduler

#sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing


#other
from IPython.core.debugger import set_trace
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
import math 
from google.colab import files
import io
import datetime as dt
import re
import pandas_profiling
import pandas_profiling as pp
from math import sqrt

#graphs
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import matplotlib.cbook as cbook
import pylab as plt
import matplotlib.dates as dates
import seaborn as sns
import pylab
import matplotlib
import matplotlib.dates
from IPython.display import display
import plotly.graph_objects as go


%matplotlib inline


# load tqdm
#!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
from tqdm import tqdm, tqdm_notebook, tnrange

device = torch.device("cuda:0" if GPU else "cpu")
device

device(type='cpu')

In [26]:
eg_input[0].shape

torch.Size([1, 3, 30])

In [27]:
# How to do padding + batching?
in_1 = prepare_input("RRWWRRRRW", padding='max_length', max_length=100)
in_2 = prepare_input("NDCQ", padding='max_length', max_length=100)
input_ids = torch.tensor([in_1, in_2], device='cuda' if GPU else 'cpu')
with torch.no_grad():
    eg_batch_input = model(input_ids)
eg_batch_input[0].shape

torch.Size([2, 100, 30])

In [219]:
# TODO: Create a special dataloaders thing that does the prepare_input func
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

class ProteinSequenceDataset(Dataset):
    def __init__(self, df, sequence_col='sequence', label_col='family_id', max_len=100):
        self.df = df.reset_index(drop=True)
        self.sequence_col = sequence_col
        self.label_col = label_col
        self.max_len = max_len
        self._label_translator = {l: i for i, l in enumerate(sorted(df[label_col].unique()))}

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        seq, label = row[self.sequence_col], row[self.label_col]
        # Convert to tensor
        seq = prepare_input(seq[:self.max_len-10], padding='max_length', max_length=self.max_len)
        seq = torch.tensor(seq, device='cuda' if GPU else 'cpu')
        label = self._label_translator[label]
        return seq, label

In [220]:
df['family_code'] = df['family_id'].cat.codes
small_df = df[df['family_id'].isin(df['family_id'].value_counts()[:3].index)]
train_df = small_df.sample(frac=0.8)
test_df = small_df.drop(train_df.index)

In [221]:
train_dataset = ProteinSequenceDataset(train_df, label_col='family_code')
test_dataset  = ProteinSequenceDataset(test_df, label_col='family_code')
train_dataset

In [222]:
from torch.utils.data import DataLoader

batch_size = 4

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

In [223]:
train_features, train_labels = next(iter(train_dataloader))
train_features, train_labels

(tensor([[ 2, 11,  5, 15, 13,  5,  5,  5, 13, 17, 23, 15, 13,  5, 12, 13,  5, 16,
          18,  5, 13, 16,  5,  3,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 2, 22,  5, 13,  9, 19,  5,  8, 23, 13, 14,  7,  7, 13,  5,  8,  6, 15,
           6,  7,  5,  9,  5, 23, 14, 14,  6,  8,  5,  5, 13, 10,  5,  6,  8,  6,
           6, 15, 22, 13,  7, 13,  7,  5,  6, 15, 13,  5,  8, 14,  6,  5,  9, 13,
          13,  6, 13,  6, 12, 13, 18, 13,  8, 21, 19,  5,  5, 15, 12, 15,  8, 16,
           7, 19, 19,  6,  6, 13,  7, 19, 13, 16,  3,  0,  0,  0,  0,  0,  0,  0,
           0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
         [ 2,  8,  5, 14,  8,  7, 10,  7, 21,  7, 16, 15, 19, 12, 20,  5,  6, 

In [224]:
for i in range(len(train_dataset)):
    x = train_dataset[i][0].shape
    if x[0] > 100:
        print(i)
        break

In [225]:
len(train_dataset.df.iloc[2120].sequence)
# x[0]

91

In [226]:
train_features.shape

torch.Size([4, 100])

In [243]:
# TODO: Create a nn class that does lstm and batching
class LSTMClassifier(nn.Module):
    """From https://stackoverflow.com/questions/47952930/how-can-i-use-lstm-in-pytorch-for-classification"""

    def __init__(self, hidden_dim, label_size, batch_size):
        self.hidden_dim = hidden_dim
        self.label_size = label_size
        self.batch_size = batch_size
        super().__init__()
        # NN part
        self.word_embeddings = model
        self.lstm = nn.LSTM(750, hidden_dim)  #, bidirectional=True
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        import torch.autograd as autograd
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        with torch.no_grad():
            embeds = self.word_embeddings(sentence)[0]
            x = embeds.view(len(sentence), self.batch_size , -1)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out[-1])
        log_probs = F.log_softmax(y, dim=-1)
        return log_probs

In [244]:
# i, data = next(enumerate(train_dataloader))
# # Every data instance is an input + label pair
# inputs, labels = data

# # Zero your gradients for every batch!
# optimizer.zero_grad()

# # Make predictions for this batch
# with torch.no_grad():
#     outputs = lstm(inputs)
# outputs

In [245]:
# lol did it work?
lstm = LSTMClassifier(2, 3, batch_size)

loss_fn = torch.nn.CrossEntropyLoss()
# TODO: Do i need to pass in that last hidden tuple?
params_to_modify = list(lstm.lstm.parameters()) + list(lstm.hidden2label.parameters())  # + list(lstm.hidden)
optimizer = torch.optim.Adam(params_to_modify, lr=0.001)


def train_one_epoch(epoch_index, verbosity=10):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(train_dataloader) instead of
    # iter(train_dataloader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, (inputs, labels) in enumerate(train_dataloader):
        # Zero your gradients for every batch!
        optimizer.zero_grad()
        # For some reason you have to do this... no idea why though
        # from https://github.com/yuchenlin/lstm_sentence_classifier/blob/master/LSTM_sentence_classifier.py#L117
        lstm.hidden = lstm.init_hidden()

        # Make predictions for this batch
        outputs = lstm(inputs)

        # Compute the loss and its gradients
        loss = loss_fn(outputs, labels)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        # Gather data and report
        running_loss += loss.item()
        if i % verbosity == verbosity-1:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            running_loss = 0.

    return last_loss

In [246]:
train_one_epoch(0, verbosity=1)

  batch 1 loss: 0.001401293158531189
  batch 2 loss: 0.001548131227493286
  batch 3 loss: 0.00180927574634552
  batch 4 loss: 0.0013589431047439575
  batch 5 loss: 0.0012338154315948487
  batch 6 loss: 0.0011785709857940673
  batch 7 loss: 0.0008708277940750122
  batch 8 loss: 0.001606055498123169
  batch 9 loss: 0.0009454374313354493
  batch 10 loss: 0.0013774211406707764
  batch 11 loss: 0.0014217284917831421


KeyboardInterrupt: ignored

In [254]:
def test_one_epoch(verbosity=10):
    with torch.no_grad():
        running_vloss = 0.0
        for i, (vinputs, vlabels) in enumerate(test_dataloader):
            voutputs = lstm(vinputs)
            vloss = loss_fn(voutputs, vlabels)
            running_vloss += vloss
            if i % verbosity == verbosity-1:
                print(f'{i=} loss test {running_vloss/(i+1):,.6f}')
        avg_vloss = running_vloss / (i + 1)
        print(f'LOSS test {avg_vloss:,.6f}')

In [255]:
test_one_epoch(verbosity=1)

i=0 loss test 0.755565
i=1 loss test 1.048644
i=2 loss test 1.124331
i=3 loss test 1.068256
i=4 loss test 1.071969


KeyboardInterrupt: ignored